### Extract text from Pdf documents and apply NER

In [1]:
import pdfplumber, os
import os.path as osp
import pandas as pd
import numpy as np
from pathlib import Path
from pdf_extract.services import file
from pdf_extract.config import global_config as glob
from pdf_extract.config import config
from pdf_extract.utils import utils
from importlib import reload
import fasttext

reload(glob)
reload(config)
reload(utils);

In [2]:
list_of_NEG_docs = os.listdir(osp.join(glob.UC_DATA_DIR, 'example_cvs', 'negatives'))
n_neg = len(list_of_NEG_docs)

list_of_POS_docs = os.listdir(osp.join(glob.UC_DATA_DIR, 'example_cvs', 'positives'))
n_pos = len(list_of_POS_docs)

print(n_pos)
print(n_neg)


13
15


Read in all Pdfs in directory:

Postives only:

In [3]:
raw_corpus1 = pd.DataFrame(columns=['text', 'fname'])

for z, fname in enumerate(list_of_POS_docs):
    i, page_objects, text = 0, {}, ""
    # The open method returns an instance of the pdfplumber.PDF class.
    with pdfplumber.open(osp.join(glob.UC_DATA_DIR, "example_cvs",f"positives/{fname}")) as pdf:
        while i < len(pdf.pages):
            page = pdf.pages[i]
            #print(pdf.metadata)
            page_objects[str(i+1)] = page.extract_text(x_tolerance=1, y_tolerance=3) #.split('\n')
            text += page_objects[str(i+1)]
            #print(f"Page {i}")
            #print(page.extract_text())
            i += 1
    
    raw_corpus1.loc[z] = [text,fname]  
raw_corpus1['label'] = 'positive'

Negatives only:

In [4]:
raw_corpus2 = pd.DataFrame(columns=['text', 'fname'])

for z, fname in enumerate(list_of_NEG_docs):
    i, page_objects, text = 0, {}, ""
    # The open method returns an instance of the pdfplumber.PDF class.
    with pdfplumber.open(osp.join(glob.UC_DATA_DIR, "example_cvs",f"negatives/{fname}")) as pdf:
        while i < len(pdf.pages):
            page = pdf.pages[i]
            page_objects[str(i+1)] = page.extract_text(x_tolerance=1, y_tolerance=3) #.split('\n')
            text += page_objects[str(i+1)]
            i += 1
    
    raw_corpus2.loc[z] = [text,fname]   
raw_corpus2['label'] = 'negative'

In [5]:
raw_corpus = pd.concat([raw_corpus1,raw_corpus2], axis=0, ignore_index=True)
#raw_corpus

Preprocess text corpus:

In [6]:
from pdf_extract.resources import preprocessor as preproc

reload(preproc)

# Preprocess corpus:
cleaner = preproc.clean_text(language='english', lemma = False, stem = False)

X_raw = raw_corpus['text'].copy()

# Full sample
X_cleaned = cleaner.fit_transform(X_raw)   

#X = raw_corpus['text']

#X_cl = cleaner.fit_transform(X_raw)

docs = X_cleaned.tolist()    

Using english language.
Using 179 stop words.
Added 8 stopword(s).
Added 13 stopword(s).
Adding custom stop words...
Setting to lower cases.
Removing whitespaces.
Applying word tokenizer.
Removing custom stopwords.
Removing punctuations.
Removing numbers.
Removing digits.
Removing non-alphabetic characters.
Removing short tokens.
Finished preprocessing!


[nltk_data] Downloading package punkt to /home/alexv84/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/alexv84/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/alexv84/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/alexv84/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
combine_fct = lambda x: (x.label == 'positive')*1

raw_corpus['text_cleaned'] = X_cleaned
raw_corpus['target'] = raw_corpus.apply(combine_fct, axis = 1)

X = raw_corpus[['target', 'text_cleaned', 'text']].reset_index(drop=True)   # keep label in X for txt-file below, will not be used directly in ft
X.target = X.target.astype(str)
#X.head()


In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X["text_cleaned"].values, X["target"].values, test_size=0.3)

print(X_train.shape)
print(X_test.shape)

(19,)
(9,)


In [20]:
from pdf_extract.utils import spacy_text_classifier as clf

reload(clf)

sclf = clf.spacy_classifier()

#sclf._fill_config_file(base_config_file_name = "base_config_textclf.cfg", config_file_name = "config.cfg")

sclf.create_docs(X_train, y_train, target_file="train.spacy")
sclf.create_docs(X_test, y_test, target_file="valid.spacy")

Pretrained spaCy model loaded.


100%|██████████| 19/19 [00:00<00:00, 44.83it/s]


/home/alexv84/Documents/Arbeit/Allianz/AZVers/data/train.spacy


100%|██████████| 9/9 [00:00<00:00, 51.81it/s]


/home/alexv84/Documents/Arbeit/Allianz/AZVers/data/valid.spacy


In [21]:
trained = sclf.fit()

Using base_config.cfg as base config file.
✔ Auto-filled config with all values
✔ Saved config
/home/alexv84/Documents/Arbeit/Allianz/AZVers/data/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy

ℹ Saving to output directory:
/home/alexv84/Documents/Arbeit/Allianz/AZVers/data/output
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'textcat']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS TEXTCAT  CATS_SCORE  SCORE 
---  ------  ------------  ------------  ----------  ------
  0       0          0.00          0.25       40.00    0.40
 10     200         44.43         28.96       41.56    0.42
 21     400          8.72          3.65       41.56    0.42
 31     600          0.00          0.00       41.56    0.42


In [14]:
sclf.predict_proba(X_test)

array([[4.45956836e-08, 1.00000000e+00],
       [1.00000000e+00, 1.58310574e-08],
       [2.15811184e-07, 9.99999762e-01],
       [3.04064951e-09, 1.00000000e+00],
       [1.04055896e-01, 8.95944059e-01],
       [9.99126256e-01, 8.73786397e-04],
       [4.04459745e-01, 5.95540285e-01],
       [4.39647152e-07, 9.99999523e-01],
       [9.98532891e-01, 1.46706449e-03]])

In [15]:
y_test

array(['1', '0', '1', '1', '1', '0', '1', '1', '0'], dtype=object)

In [17]:
[str(i) for i in sclf.predict(X_test)]

['1', '0', '1', '1', '1', '0', '1', '1', '0']

In [21]:
sclf.score(X = X_test, y = y_test)

0.3333333333333333

In [19]:
from sklearn.metrics import classification_report

print(classification_report(y_train, [str(i) for i in sclf.predict(X_train)], target_names=["0","1"]))
print(classification_report(y_test, [str(i) for i in sclf.predict(X_test)], target_names=["0","1"]))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        12
           1       1.00      1.00      1.00         7

    accuracy                           1.00        19
   macro avg       1.00      1.00      1.00        19
weighted avg       1.00      1.00      1.00        19

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         3
           1       1.00      1.00      1.00         6

    accuracy                           1.00         9
   macro avg       1.00      1.00      1.00         9
weighted avg       1.00      1.00      1.00         9



In [9]:
from spacy.tokens import DocBin
import spacy
from tqdm import tqdm

nlp = spacy.load("de_core_news_lg") 
#nlp = spacy.load("de_core_web_lg")     
#nlp = spacy.blank('en')              # blank model; to train from scratch on own corpus

nlp.pipe_names

['tok2vec',
 'tagger',
 'morphologizer',
 'parser',
 'lemmatizer',
 'attribute_ruler',
 'ner']

In [10]:
from typing import Set, List, Tuple
from spacy.tokens import DocBin

def create_docs(data: List[Tuple[str, str]], target_file: str, cats: Set[str]):
    
    docs = DocBin()
    for doc, label in nlp.pipe(data, as_tuples=True):
        for cat in cats:
            doc.cats[cat] = 1 if cat == label else 0
        docs.add(doc)
    docs.to_disk(os.path.join(glob.UC_DATA_DIR, target_file))
    print(os.path.join(glob.UC_DATA_DIR, target_file))
    return docs

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X["text_cleaned"].values, X["target"].values, test_size=0.3)

print(X_train.shape)
print(X_test.shape)

cats = ["0","1"]

create_docs(list(zip(X_train, y_train)), "train.spacy", cats=cats)
create_docs(list(zip(X_test, y_test)), "valid.spacy", cats=cats)

(19,)
(9,)
/home/alexv84/Documents/Arbeit/Allianz/AZVers/data/train.spacy
/home/alexv84/Documents/Arbeit/Allianz/AZVers/data/valid.spacy


In [12]:
# import subprocess

# base_config_file_name = "base_config_textclf.cfg"
# config_file_name = "config.cfg"

# cmd_init = 'python -m spacy init fill-config {}/{} {}/{}'.format(glob.UC_DATA_DIR, base_config_file_name, glob.UC_DATA_DIR, config_file_name)

# process = subprocess.Popen(cmd_init.split(), stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
# stdout_cmd, stderr_cmd = process.communicate()
# print(stdout_cmd.decode("utf-8"))     # convert bytes to string for nicer printing

In [13]:
!python -m spacy init fill-config /home/alexv84/Documents/Arbeit/Allianz/AZVers/data/base_config_textclf.cfg /home/alexv84/Documents/Arbeit/Allianz/AZVers/data/config.cfg   

✔ Auto-filled config with all values
✔ Saved config
/home/alexv84/Documents/Arbeit/Allianz/AZVers/data/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [14]:
from spacy.cli.train import train as train_model

train_model(config_path = f"{glob.UC_DATA_DIR}/config.cfg",
        output_path = f"{glob.UC_DATA_DIR}/output",
        overrides={"paths.train": f"{glob.UC_DATA_DIR}/train.spacy",
                "paths.dev": f"{glob.UC_DATA_DIR}/valid.spacy",
    },
)

print("\nTraining done!")

ℹ Saving to output directory:
/home/alexv84/Documents/Arbeit/Allianz/AZVers/data/output
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'textcat']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS TEXTCAT  CATS_SCORE  SCORE 
---  ------  ------------  ------------  ----------  ------
  0       0          0.00          0.25       35.71    0.36
 10     200         90.71         20.48       77.50    0.77
 21     400         41.17         12.40       35.71    0.36
 31     600        229.46          7.70       35.71    0.36
 42     800        324.93         20.71       50.00    0.50
 52    1000          0.00          0.00       50.00    0.50
 63    1200          0.00          0.00       50.00    0.50
 73    1400          0.00          0.00       50.00    0.50
 84    1600          0.00          0.00       50.00    0

In [27]:
# Use best model fit:
trained_nlp = spacy.load(f"{glob.UC_DATA_DIR}/output/model-best") 


In [56]:
id = 2

text = X['text_cleaned'][id]
#print(text)
print(X['target'][id])

# Perform the trained pipeline on this text
doc = trained_nlp(text)

scores = doc.cats['0'], doc.cats['1']
scores
#predicted_class = np.array(scores).argmax(axis=1)
#predicted_class

gabriel kordon kordon mangfallstraße rosenheim persönliche daten name gabriel kordon geboren aibling aktueller wohnort mangfallstraße rosenheim kontaktinformation mobil mail kordon schul berufsausbildung ende technische universität münchen angestrebter abschluss mathematik master mixed poisson model lassa fever disease nigeria technische naturwissenschaftliche universität norwegens ntnu erasmus student abschluss mathematik bachelorarbeit regressionsanalyse ignaz günther gymnasium rosenheim abschluss allgemeine hochschulreife jobs praktika sept kpmg werkstudent bereich data science financial services cfro insurance projektarbeit einem großen deutschen versicherer münchener verein werkstudent bereich life insurance praktikum aschemann sternstunden catering service tender host expo real seit jahren rosenheim raubling tennistraining angehender trainer leistungssport schreinerei marschner einwöchiges praktikum ehrenamt seit jugendwart rosenheim besondere fähigkeiten fundierte kenntnisse com

(9.196549077827854e-11, 1.0)

In [60]:
query = X['text_cleaned'].head(2).tolist()
#query

In [68]:
#trained_nlp.tokenizer(text)

scores = [trained_nlp(text).cats for text in query]
scores


[{'0': 2.180798944206863e-12, '1': 1.0},
 {'0': 0.9999996423721313, '1': 3.010082991750096e-07}]

In [75]:
pd.DataFrame(scores)


,0,1
0,2.180799e-12,1.000000e+00
1,9.999996e-01,3.010083e-07


In [76]:
predicted_class = pd.DataFrame(scores).values.argmax(axis=1)
predicted_class

array([1, 0])

In [30]:
doc_from_pretrained = nlp(text)

print("entities:", doc_from_pretrained.ents)
print("sentences:", list(doc_from_pretrained.sents))
print("classification:", doc_from_pretrained.cats)    # empty -> add trained_nlp pipeline!

In [32]:
#nlp = spacy.load("de_core_news_lg", exclude=["textcat"])
#nlp.add_pipe("textcat", source = trained_nlp)

In [41]:
spacy.explain("ORG")

'Companies, agencies, institutions, etc.'

In [42]:
spacy.explain("GPE")

'Countries, cities, states'

In [44]:
#displacy.render(text1,style="ent",jupyter=True)

In [1]:
from bs4 import BeautifulSoup
import requests
import re

In [28]:
URL="https://www.zeebiz.com/markets/currency/news-cryptocurrency-news-today-june-12-bitcoin-dogecoin-shiba-inu-and-other-top-coins-prices-and-all-latest-updates-158490"

In [29]:
html_content = requests.get(URL).text

In [30]:
soup = BeautifulSoup(html_content, "lxml")

In [31]:
body=soup.body.text

In [32]:
body= body.replace('\n', ' ')
#body= body.replace('t', ' ')
#body= body.replace('r', ' ')
#body= body.replace('xa0', ' ')
#body=re.sub(r'[^ws]', '', body)

In [148]:
#body

In [149]:
text3= NER(body)